# Домашнее задание 1. Извлечение ключевых слов

При выполнении домашнего задания можно пользоваться материалами лекций и семинаров.

### Описание задания

1. (1 балл) Подготовить мини-корпус (не меньше 4 текстов, примерный общий объём - 3-5 тысяч токенов) с разметкой ключевых слов. 
Предполагается, что вы найдете источник текстов, в котором **уже выделены** ключевые слова.
Укажите источник корпуса и опишите, в каком виде там были представлены ключевые слова.

2. (2 балла) Разметить ключевые слова самостоятельно. Оценить пересечение с имеющейся разметкой.
Составить эталон разметки (например, пересечение или объединение вашей разметки и исходной).

3. (2 балла) Применить к этому корпусу 3 метода извлечения ключевых слов на выбор (RAKE, TextRank, tf*idf, OKAPI BM25, ...)

4. (2 балла) Составить морфологические/синтаксические шаблоны для ключевых слов и фраз, выделить соответствующие им подстроки из корпуса (например, именные группы Adj+Noun).
Применить эти фильтры к спискам ключевых слов.

4. (2  балла) Оценить точность, полноту, F-меру выбранных методов относительно эталона:
с учётом морфосинтаксических шаблонов и без них.

5. (1 балл) Описать ошибки автоматического выделения ключевых слов (что выделяется лишнее, что не выделяется);
предложить свои методы решения этих проблем.

In [3]:
# !pip install keybert
# !pip install pymorphy2
# !pip install pymorphy2-dicts-uk

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 708.0 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for keybert: filename=keybert-0.7.0-py3-none-any.whl size=23799 sha256=0007d92ed2632161433bea7096acfb516e34133be315186c69a951173815e8af
  Stored in directory: /root/.cache/pip/wheels/85/0d/12/77d219f3ebbb22dc22234b4d665886c0eace86a26eca0aa72b
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=98a5bcf9ce2fb33569f2b76171280b0b70d91118363ef8641a83f8317f3b97bc
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built keybert sentence-transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 457.1 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 13.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0

In [149]:
from __future__ import annotations

from string import punctuation
from tqdm import tqdm

import pandas as pd
import numpy as np

import spacy
import pymorphy2
from keybert import KeyBERT
from nltk.corpus import stopwords
from nltk import tokenize

morph = pymorphy2.MorphAnalyzer(lang='uk')
lemm = spacy.load('en_core_web_sm')
stopwords_eng = stopwords.words("english")
punctuation += '—…«»'

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

# 1. Создание корпуса

(1 балл) Подготовить мини-корпус (не меньше 4 текстов, примерный общий объём - 3-5 тысяч токенов) с разметкой ключевых слов. Предполагается, что вы найдете источник текстов, в котором уже выделены ключевые слова. Укажите источник корпуса и опишите, в каком виде там были представлены ключевые слова.

**Источник текстов:** [соревнование по Keyword Extraction от FaceBook](https://www.kaggle.com/competitions/facebook-recruiting-iii-keyword-extraction). Из исходного train датасета были взять рандомные 20 текстов, общая сумма токенов в которых составляет 3 148. Ключевые слова помечены как тэги.  

## 1.1 Корпус

In [150]:
df = pd.read_csv('../input/nlp-hw1-dataset/Train_sample.csv')
df.head()

,Unnamed: 0,Title,Body,Tags
0,916626,I dont understand why this log4j.xml is wrong,<p>I wanna log into cassandra db with log4j.</...,log4j cassandra
1,768258,clone git repository via active FTP,<p>I'm a branch office worker and have uploade...,git ftp clone
2,2368609,How to print XSLT version supported by Xalan i...,<p>I am using Xalan C++ library and I want to ...,xalan
3,131806,Hibernate mapping for cyclic relation ships,<p>I am learning a hibernate and I am developi...,hibernate
4,4213547,ffmpeg: how to add cartoon effect?,<p>I was wondering if anyone out there has use...,ffmpeg


In [151]:
print('Сумма токенов:', df['Body'].apply(lambda x: len(x.split())).sum())

Сумма токенов: 3148


## 1.2 Preprocessing

In [152]:
def remove_punctuation(text: str) -> str:
    for i in punctuation:
        text = text.replace(i, ' ')
    return text


def lemmatization(text: str) -> list:
    return [token.lemma_ for token in lemm(text)]


def remove_stopwords(text: list) -> str:
    clean_text = [word for word in text if (word not in stopwords_eng) and (word != ' ')]
    return ' '.join(clean_text)


def preprocessing(text: str):
    lemm_text = remove_punctuation(text)
    lemm_text = lemmatization(lemm_text)
    lemm_text = remove_stopwords(lemm_text)
    return lemm_text

In [153]:
df['Body'] = df['Body'].apply(preprocessing)

# 2. Cамостоятельная разметка ключевых слов

(2 балла) Разметить ключевые слова самостоятельно. Оценить пересечение с имеющейся разметкой. Составить эталон разметки (например, пересечение или объединение вашей разметки и исходной).

In [154]:
fb_tags = [i.split() for i in df['Tags']]
my_tags = [
    ['log4j', 'cassandra'], 
    ['ftp', 'server', 'git', 'clone', 'password'], 
    ['xalan', 'c++', 'xsl'], 
    ['hibernate', 'dependency'], 
    ['frie0r', 'ffmpeg'], 
    ['php', 'javascript', 'jquery', 'ajax', 'fwrite'], 
    ['github', 'bespin'], 
    ['bzip2', 'EOF'], 
    ['javadocs'], 
    ['zip'], 
    ['software',  'ghost'], 
    ['transactions', 'R'], 
    ['hdb2ddl', 'hibernate'], 
    ['ajax', 'php', 'mysql', 'post'], 
    ['varchar', 'bigint'], 
    ['php', 'counter'], 
    ['xml', 'flash'], 
    ['language', 'search'], 
    ['array'], 
    ['location', 'android', 'map', 'API']
]

In [155]:
intercept = 0
for tag1, tag2 in zip(fb_tags, my_tags):
    if set(tag1) == set(tag2):
        intercept += 1
print(f'Разметка совпадает в {intercept} строках из {len(my_tags)} строк.')

Разметка совпадает в 5 строках из 20 строк.


In [156]:
gold_tags = [list(set(tag1) | set(tag2)) for tag1, tag2 in zip(fb_tags, my_tags)]
gold_tags

[['cassandra', 'log4j'],
 ['password', 'server', 'clone', 'git', 'ftp'],
 ['c++', 'xsl', 'xalan'],
 ['hibernate', 'dependency'],
 ['frie0r', 'ffmpeg'],
 ['ajax', 'fwrite', 'jquery', 'javascript', 'php'],
 ['github', 'bespin'],
 ['EOF', 'bzip2'],
 ['java', 'javadocs', 'javadoc'],
 ['zip'],
 ['ghost', 'software'],
 ['r', 'transactions', 'R'],
 ['hdb2ddl', 'java', 'hibernate'],
 ['ajax', 'mysql', 'php', 'post'],
 ['bigint', 'varchar', 'sql-server-2005'],
 ['counter', 'php'],
 ['flash', 'xml'],
 ['language', 'search', 'sql'],
 ['arrays', 'c', 'array'],
 ['map', 'location', 'API', 'android']]

# 3. Извлечение ключевых слов

(2 балла) Применить к этому корпусу 3 метода извлечения ключевых слов: RAKE, TextRank, tf*idf, OKAPI BM25

In [157]:
n = 5

# 3.1 TF*IDF

In [158]:
def calculate_tfidf(texts: list) -> list[list[str]]:
    tfidf_vectorizer = TfidfVectorizer(use_idf=True, norm='l2')
    matrix = tfidf_vectorizer.fit_transform(texts).toarray()
    vocabulary = tfidf_vectorizer.get_feature_names_out()
    
    tags, values = [], []
    index2token = {index: token for token, index in (tfidf_vectorizer.vocabulary_).items()}
    for row in matrix:
        max_args = [arg for arg in (-row).argsort()[:n]]
        max_tokens = [index2token[arg] for arg in max_args]
        max_values = np.take(row, max_args)
        tags.append(max_tokens)
        values.append(max_values)
    return tags, values

In [159]:
tags_tfidf, values_tfidf = calculate_tfidf(df['Body'])
tags_tfidf

[['log4j', 'gt', 'lt', 'name', 'cassandra'],
 ['ftp', 'git', 'company', 'repo', 'com'],
 ['xalan', 'support', 'version', 'xsl', 'print'],
 ['address', 'person', 'private', 'class', 'model'],
 ['cartoon', 'output', 'look', 'openmovieeditor', 'filtereffect'],
 ['gt', 'lt', 'type', 'script', 'value'],
 ['bespin', 'https', 'mozillalab', 'mozilla', 'vcsintegration'],
 ['bzip2', 'limit', 'block', 'bzip2recover', 'eof'],
 ['search', 'id', 'tag', 'javadoc', 'frame'],
 ['zip', 'directory', 'p1', 'unzip', 'subdirectory'],
 ['software', 'backup', 'imaging', 'question', 'http'],
 ['code', 'transaction', 'apriori', '2322', 'algorithm'],
 ['gt', 'lt', 'public', 'classeb', 'code'],
 ['completely', 'post', 'file', 'finish', 'ajax'],
 ['datatype', 'phone', 'store', '2005', 'sql'],
 ['colcount', 'mysql', 'result', 'artwork', 'row'],
 ['url', 'urlxml', 'trace', 'xml', 'success'],
 ['etc', 'language', 'establish', 'side', 'category'],
 ['code', 'value', 'array', 'em', 'element'],
 ['location', 'android', 

## 3.2 OKAPI BM25

In [160]:
def calculate_bm25(texts: list, k: int = 2, b: int = 0.75) -> list[list[str]]:
    # tf
    count_vectorizer = CountVectorizer()
    count = count_vectorizer.fit_transform(texts).toarray()
    tf = count

    # idf
    tfidf_vectorizer = TfidfVectorizer(use_idf=True, norm='l2')
    tfidf = tfidf_vectorizer.fit_transform(texts).toarray()
    vocabulary = tfidf_vectorizer.get_feature_names_out()
    idf = tfidf_vectorizer.idf_
    idf = np.expand_dims(idf, axis=0)

    # расчет количества слов в каждом документе - l(d)
    len_d = tf.sum(axis=1)
    
    # расчет среднего количества слов документов корпуса - avdl
    avdl = len_d.mean()

    # расчет числителя
    A = idf * tf * (k + 1)

    # расчет знаменателя
    B_1 = (k * (1 - b + b * len_d / avdl))
    B_1 = np.expand_dims(B_1, axis=-1)
    B = tf + B_1

    # BM25
    matrix = A / B
    
    # get tags
    index2token = {index: token for token, index in (count_vectorizer.vocabulary_).items()}
    tags = []
    values = []
    for row in matrix:
        max_args = [arg for arg in (-row).argsort()[:n]]
        max_tokens = [index2token[arg] for arg in max_args]
        max_values = np.take(row, max_args)
        tags.append(max_tokens)
        values.append(max_values)
    return tags, values

In [161]:
tags_bm25, values_bm25 = calculate_bm25(df['Body'])
tags_bm25

[['log4j', 'appender', 'cassandra', 'param', 'jdbc'],
 ['ftp', 'repo', 'company', 'git', 'username'],
 ['xalan', 'xsl', 'print', 'support', 'version'],
 ['address', 'person', 'dependency', 'private', 'model'],
 ['cartoon', 'openmovieeditor', 'ffmpeg', 'video', 'filtereffect'],
 ['script', 'text', 'type', 'gt', 'value'],
 ['bespin', 'https', 'mozillalab', 'mozilla', 'vcsintegration'],
 ['bzip2', 'limit', 'block', 'bzip2recover', 'eof'],
 ['id', 'tag', 'javadoc', 'search', 'frame'],
 ['zip', 'directory', 'ignore', '7z', 'subdirectory'],
 ['software', 'backup', 'imaging', 'installation', '2010'],
 ['transaction', '2322', 'apriori', 'algorithm', '1141'],
 ['classeb', 'integer', 'cb', 'annotazione', 'numero'],
 ['finish', 'completely', 'post', 'ajax', 'run'],
 ['datatype', 'phone', 'store', '2005', 'sql'],
 ['colcount', 'artwork', 'mysql', 'result', 'cell'],
 ['urlxml', 'trace', 'url', 'success', 'flash'],
 ['language', 'establish', 'side', 'category', 'define'],
 ['array', 'em', 'value', '

## 3.3 Keybert

In [162]:
kw_model = KeyBERT('clips/mfaq')

/opt/conda/lib/python3.7/site-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


In [163]:
def calculate_kbert(texts: list) -> list[list[str]]:
    tags = []
    values = []
    texts_kw = kw_model.extract_keywords(texts)
    for text_kw in texts_kw:
        keywords = [word for word, prob in text_kw]
        prob = [prob for word, prob in text_kw]
        tags.append(keywords)
        values.append(prob)
    return tags, values

In [164]:
tags_kbert, values_kbert = calculate_kbert(df['Body'])
tags_kbert

[['logg', 'configuration', 'cql', 'log', 'logging'],
 ['ftp', 'server', 'install', 'microsoft', 'command'],
 ['support', 'xsl', 'version', 'want', 'xslt'],
 ['learn', 'project', 'class', 'mapping', 'develop'],
 ['exe', 'ffmpeg', 'openmovieeditor', 'video', 'cartoon'],
 ['execute', 'config', 'function', 'xmlhttprequest', 'xhtml'],
 ['github', 'git', 'vcsintegration', 'mozillalab', 'wiki'],
 ['error', 'modify', 'problem', 'eof', 'bzip2'],
 ['code', 'search', 'documentation', 'add', 'tag'],
 ['unzip', 'tool', 'code', 'command', 'file'],
 ['backup', 'blockquote', 'software', 'like', 'disk'],
 ['code', 'read', 'try', 'function', 'tr'],
 ['string', 'setnumero', 'setcb', 'encode', 'cb'],
 ['php', 'task', 'user', 'post', 'download'],
 ['sql', 'blockquote', 'datatype', 'href', 'example'],
 ['syntax', 'mysql', 'add', 'code', 'php'],
 ['xmlurl', 'urlxml', 'xml', 'figure', 'stackoverflow'],
 ['sql', 'logical', 'define', 'definition', 'googles'],
 ['array', 'initialize', 'element', 'build', 'solve'

# 4. Составить шаблоны

(2 балла) Составить морфологические/синтаксические шаблоны для ключевых слов и фраз, выделить соответствующие им подстроки из корпуса (например, именные группы Adj+Noun). Применить эти фильтры к спискам ключевых слов.

In [165]:
def text_to_tokens(text: str) -> list[list[str]]:
    return [tokenize.word_tokenize(sentence) for sentence in tokenize.sent_tokenize(text)]


def tokens_to_pos(tokens: list[list[str]]) -> list[list[str]]:
    poses = []
    for sentence in tokens:
        sent_poses = []
        for token in sentence:
            sent_poses.append(lemm(token)[0].pos_)
        poses.append(sent_poses)
    return poses

    
def extract_template(text: str, first_pos: str = "ADJ", second_pos: str = "NOUN") -> str:
    tokens = text_to_tokens(text)
    poses = tokens_to_pos(tokens)
    templates = []
    for i in range(len(poses)):
        sent_tokens = tokens[i]
        sent_pos = poses[i]
        sent_templates = []
        for j in range(len(sent_pos) - 1):
            if sent_pos[j] == first_pos and sent_pos[j+1] == second_pos:
                sent_templates.append(sent_tokens[j])
                sent_templates.append(sent_tokens[j+1])
        templates.append(sent_templates)
    return ".".join([" ".join(sent) for sent in templates])

In [166]:
df['Adj+Noun'] = df['Body'].apply(lambda x: extract_template(x, first_pos='ADJ', second_pos='NOUN'))
df['Noun+Noun'] = df['Body'].apply(lambda x: extract_template(x, first_pos='NOUN', second_pos='NOUN'))

In [167]:
def get_best_tags(tags_NN, values_NN, tags_AN, values_AN) -> list[list[str]]:
    best_tokens = []
    for t_nn, t_an, v_nn, v_an in zip(tags_NN, tags_AN, 
                                      values_NN, values_AN):
        concat_v = np.concatenate((v_nn, v_an), axis=0)
        concat_t = t_nn + t_an
        max_args = [arg for arg in (-concat_v).argsort()[:n]]
        best_tokens.append(list(np.take(concat_t, max_args)))
    return best_tokens

In [168]:
# TF-IDF
tags_tfidf_NN, values_tfidf_NN = calculate_tfidf(df['Noun+Noun'])
tags_tfidf_AN, values_tfidf_AN = calculate_tfidf(df['Adj+Noun'])
tags_tfidf_templ = get_best_tags(tags_tfidf_NN, values_tfidf_NN, tags_tfidf_AN, values_tfidf_AN)

# BM25
tags_bm25_NN, values_bm25_NN = calculate_bm25(df['Noun+Noun'])
tags_bm25_AN, values_bm25_AN = calculate_bm25(df['Adj+Noun'])
tags_bm25_templ = get_best_tags(tags_bm25_NN, values_bm25_NN, tags_bm25_AN, values_bm25_AN)

# KeyBERT
tags_kbert_NN, values_kbert_NN = calculate_kbert(df['Noun+Noun'])
tags_kbert_AN, values_kbert_AN = calculate_kbert(df['Adj+Noun'])
tags_kbert_templ = get_best_tags(tags_kbert_NN, values_kbert_NN, tags_kbert_AN, values_kbert_AN)

# 5. Точность, полнота, F-мера

(2 балла) Оценить точность, полноту, F-меру выбранных методов относительно эталона: с учётом морфосинтаксических шаблонов и без них.

$$ Precision =  \frac{|correctly\;extracted\;entities|}  {|entities\;extracted\;by\;model|} $$


$$ Recall =  \frac{|correctly\;extracted\;entities|}  {|entities\;that\;are\;keywords|} $$


$$ F1 =  \frac{2*Precision*Recall}  {Precision + Recall} $$

In [169]:
def precision(pred_entities: list, gold_entities: list) -> float:
    corr = 0
    all_ent = 0
    
    for pred, gold in zip(pred_entities, gold_entities):
        corr += (len(set(pred) & set(gold)))
        all_ent += len(pred)
    return round(corr / all_ent, 4)

        
def recall(pred_entities: list, gold_entities: list) -> float:
    corr = 0
    all_ent = 0
    
    for pred, gold in zip(pred_entities, gold_entities):
        corr += (len(set(pred) & set(gold)))
        all_ent += len(gold)
    return round(corr / all_ent, 4)


def f1(pred: list, gold: list) -> float:
    p = precision(pred, gold)
    r = recall(pred, gold)
    return round((2 * p * r) / (p + r), 4)

## 5.1 Без учета шаблонов
### 5.1.1 TF-IDF

In [170]:
print('Precision:', precision(tags_tfidf, gold_tags))
print('Recall:', recall(tags_tfidf, gold_tags))
print('F1:', f1(tags_tfidf, gold_tags))

Precision: 0.19
Recall: 0.3393
F1: 0.2436


### 5.1.2 BM25

In [171]:
print('Precision:', precision(tags_bm25, gold_tags))
print('Recall:', recall(tags_bm25, gold_tags))
print('F1:', f1(tags_bm25, gold_tags))

Precision: 0.2
Recall: 0.3571
F1: 0.2564


### 5.1.3 KeyBERT

In [172]:
print('Precision:', precision(tags_kbert, gold_tags))
print('Recall:', recall(tags_kbert, gold_tags))
print('F1:', f1(tags_kbert, gold_tags))

Precision: 0.13
Recall: 0.2321
F1: 0.1667


## 5.2 С учетом шаблонов
### 5.2.1 TF-IDF

In [173]:
print('Precision:', precision(tags_tfidf_templ, gold_tags))
print('Recall:', recall(tags_tfidf_templ, gold_tags))
print('F1:', f1(tags_tfidf_templ, gold_tags))

Precision: 0.1
Recall: 0.1786
F1: 0.1282


### 5.2.2 BM25

In [174]:
print('Precision:', precision(tags_bm25_templ, gold_tags))
print('Recall:', recall(tags_bm25_templ, gold_tags))
print('F1:', f1(tags_bm25_templ, gold_tags))

Precision: 0.1
Recall: 0.1786
F1: 0.1282


### 5.2.3 KeyBERT

In [175]:
print('Precision:', precision(tags_kbert_templ, gold_tags))
print('Recall:', recall(tags_kbert_templ, gold_tags))
print('F1:', f1(tags_kbert_templ, gold_tags))

Precision: 0.1122
Recall: 0.1964
F1: 0.1428


# 6. Ошибки автоматического выделения ключевых слов

(1 балл) Описать ошибки автоматического выделения ключевых слов (что выделяется лишнее, что не выделяется); предложить свои методы решения этих проблем.

Я выбрала достаточно специфичный датасет, содержащий вопросы из StackOverflow. В вопросах часто содержатся не только вопрос и описание проблемы, но и куски кода, что заметно влияет на работу KeyWord Extraction моделей. 

Например, **tf-idf** и **bm25** часто выделяют как ключевые слова _print, import_  

С одной стороны, может иметь смысл на шаге предобработки удалять куски с кодом, потому что, по интуиции, ключевые слова содержатся с вопросе. С друго стороны, и в коде могут содержаться ключевые слова, например импортируемая бибилиотека или модель. 

Кроме того, модели часто выделяют цифры как ключевые слова (в основном это год или случайный набор цифр). Решение: на этапе предобработки удалять цифры, которые не входят в состав слов (например, оставляем _2_ в _pymorphy2_, но удаляем _2022_).

In [176]:
def false_positive(pred_tags: list, gold_tags: list, model: str) ->list:
    print(f'Keywords ошибочно предложенные моделью {model}\n')
    fp = [set(pred) - set(gold) for pred, gold in zip(pred_tags, gold_tags)]
    return fp


def false_negative(pred_tags: list, gold_tags: list, model: str) ->list:
    print(f'Keywords ошибочно НЕ предложенные моделью {model}\n')
    fn = [set(gold) - set(pred) for pred, gold in zip(pred_tags, gold_tags)]
    return fn

In [177]:
false_positive(tags_tfidf, gold_tags, model='tf-idf')

Keywords ошибочно предложенные моделью tf-idf



[{'gt', 'lt', 'name'},
 {'com', 'company', 'repo'},
 {'print', 'support', 'version'},
 {'address', 'class', 'model', 'person', 'private'},
 {'cartoon', 'filtereffect', 'look', 'openmovieeditor', 'output'},
 {'gt', 'lt', 'script', 'type', 'value'},
 {'https', 'mozilla', 'mozillalab', 'vcsintegration'},
 {'block', 'bzip2recover', 'eof', 'limit'},
 {'frame', 'id', 'search', 'tag'},
 {'directory', 'p1', 'subdirectory', 'unzip'},
 {'backup', 'http', 'imaging', 'question'},
 {'2322', 'algorithm', 'apriori', 'code', 'transaction'},
 {'classeb', 'code', 'gt', 'lt', 'public'},
 {'completely', 'file', 'finish'},
 {'2005', 'datatype', 'phone', 'sql', 'store'},
 {'artwork', 'colcount', 'mysql', 'result', 'row'},
 {'success', 'trace', 'url', 'urlxml'},
 {'category', 'establish', 'etc', 'side'},
 {'code', 'element', 'em', 'value'},
 {'gms', 'import'}]

In [178]:
false_negative(tags_tfidf, gold_tags, model='tf-idf')

Keywords ошибочно НЕ предложенные моделью tf-idf



[set(),
 {'clone', 'password', 'server'},
 {'c++'},
 {'dependency', 'hibernate'},
 {'ffmpeg', 'frie0r'},
 {'ajax', 'fwrite', 'javascript', 'jquery', 'php'},
 {'github'},
 {'EOF'},
 {'java', 'javadocs'},
 set(),
 {'ghost'},
 {'R', 'r', 'transactions'},
 {'hdb2ddl', 'hibernate', 'java'},
 {'mysql', 'php'},
 {'bigint', 'sql-server-2005', 'varchar'},
 {'counter', 'php'},
 {'flash'},
 {'search', 'sql'},
 {'arrays', 'c'},
 {'API'}]

In [179]:
false_positive(tags_bm25, gold_tags, model='bm25')

Keywords ошибочно предложенные моделью bm25



[{'appender', 'jdbc', 'param'},
 {'company', 'repo', 'username'},
 {'print', 'support', 'version'},
 {'address', 'model', 'person', 'private'},
 {'cartoon', 'filtereffect', 'openmovieeditor', 'video'},
 {'gt', 'script', 'text', 'type', 'value'},
 {'https', 'mozilla', 'mozillalab', 'vcsintegration'},
 {'block', 'bzip2recover', 'eof', 'limit'},
 {'frame', 'id', 'search', 'tag'},
 {'7z', 'directory', 'ignore', 'subdirectory'},
 {'2010', 'backup', 'imaging', 'installation'},
 {'1141', '2322', 'algorithm', 'apriori', 'transaction'},
 {'annotazione', 'cb', 'classeb', 'integer', 'numero'},
 {'completely', 'finish', 'run'},
 {'2005', 'datatype', 'phone', 'sql', 'store'},
 {'artwork', 'cell', 'colcount', 'mysql', 'result'},
 {'success', 'trace', 'url', 'urlxml'},
 {'category', 'define', 'establish', 'side'},
 {'1024', 'em', 'iteration', 'value'},
 {'gms', 'import', 'loc'}]

In [180]:
false_negative(tags_bm25, gold_tags, model='bm25')

Keywords ошибочно НЕ предложенные моделью bm25



[set(),
 {'clone', 'password', 'server'},
 {'c++'},
 {'hibernate'},
 {'frie0r'},
 {'ajax', 'fwrite', 'javascript', 'jquery', 'php'},
 {'github'},
 {'EOF'},
 {'java', 'javadocs'},
 set(),
 {'ghost'},
 {'R', 'r', 'transactions'},
 {'hdb2ddl', 'hibernate', 'java'},
 {'mysql', 'php'},
 {'bigint', 'sql-server-2005', 'varchar'},
 {'counter', 'php'},
 {'xml'},
 {'search', 'sql'},
 {'arrays', 'c'},
 {'API', 'map'}]